<h1>Importaciones

In [3]:
import pandas as pd
import os
import utils
import pyarrow as pa
import pyarrow.parquet as pq
import warnings
warnings.filterwarnings("ignore")

<h1>ETL

Basados en los resultados del [diagnóstico](diagnostico.ipynb), los archivos business.pkl, tip.json, y user.parquet requieren transformación importante, mientras que checkin.json y review.json solo requieren revisión y filtrado.

<h2>Business

Cargamos el archivo business.pkl a un DataFrame.

In [5]:
businessDf = utils.loadFile('../data/Yelp/business.pkl')
businessDf.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Eliminamos las columnas duplicadas.

In [6]:
# Extraemos columnas duplicadas en otro dataframe
businessDuppedDf = businessDf.loc[:, businessDf.columns.duplicated()]
# Eliminamos columnas duplicadas del dataframe original
businessDf = businessDf.loc[:, ~businessDf.columns.duplicated()]
# Eliminamos filas vacías del dataframe de columnas duplicadas
businessDuppedDf = businessDuppedDf.dropna(axis=0, how='all')


Revisamos el dataframe duplicado.

In [7]:
businessDuppedDf

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
590,6_Zf2QsRHCH1RhbzA9inyw,STarbucks,8602 Hillsborough Ave,Tampa,LA,33615,27.99637,-82.580954,3.0,56,1,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Coffee & Tea, Food","{'Monday': '5:30-21:0', 'Tuesday': '5:30-21:0'..."
613,Iipfa9Kaac9XH5Vo8GQwvg,STarbucks,5956 153 Avenue,Edmonton,PA,T5Y 2W1,53.616147,-113.432424,3.5,10,1,"{'BusinessParking': '{'garage': False, 'street...","Coffee & Tea, Food","{'Monday': '5:30-22:0', 'Tuesday': '5:30-22:0'..."
978,oY6osC4Ch3PvguA7D1hh_w,STarbucks,1251 Route 130 S,Cinnaminson,AZ,08077,40.000157,-74.987313,4.5,7,1,"{'WiFi': 'u'free'', 'BusinessParking': '{'gara...","Coffee & Tea, Food","{'Monday': '5:30-20:0', 'Tuesday': '5:30-20:0'..."
1296,pxuMxh5oC7W-PDN9wwvozg,STarbucks,219 N Center St,Reno,FL,89501,39.527772,-119.812518,3.0,44,0,"{'BikeParking': 'False', 'Caters': 'False', 'O...","Coffee & Tea, Food","{'Monday': '5:30-19:0', 'Tuesday': '5:30-19:0'..."
1543,lo2VbA7Lwzu3i23pQpmzlw,STarbucks,"1125 S Black Horse Pike, Space 575",Gloucester Township,NJ,08012,39.78403,-75.05207,3.0,15,1,"{'BikeParking': 'True', 'RestaurantsPriceRange...","Food, Coffee & Tea","{'Monday': '0:0-0:0', 'Tuesday': '10:30-20:0',..."


Vemos que los datos de businessDuppedDf ya se encuentran en businessDf por lo que se descartan.

In [8]:
businessDf[businessDf['business_id'].isin(businessDuppedDf['business_id'].unique())]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
590,6_Zf2QsRHCH1RhbzA9inyw,Starbucks,8602 Hillsborough Ave,Tampa,LA,33615,27.99637,-82.580954,3.0,56,1,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Coffee & Tea, Food","{'Monday': '5:30-21:0', 'Tuesday': '5:30-21:0'..."
613,Iipfa9Kaac9XH5Vo8GQwvg,Starbucks,5956 153 Avenue,Edmonton,PA,T5Y 2W1,53.616147,-113.432424,3.5,10,1,"{'BusinessParking': '{'garage': False, 'street...","Coffee & Tea, Food","{'Monday': '5:30-22:0', 'Tuesday': '5:30-22:0'..."
978,oY6osC4Ch3PvguA7D1hh_w,Starbucks,1251 Route 130 S,Cinnaminson,AZ,08077,40.000157,-74.987313,4.5,7,1,"{'WiFi': 'u'free'', 'BusinessParking': '{'gara...","Coffee & Tea, Food","{'Monday': '5:30-20:0', 'Tuesday': '5:30-20:0'..."
1296,pxuMxh5oC7W-PDN9wwvozg,Starbucks,219 N Center St,Reno,FL,89501,39.527772,-119.812518,3.0,44,0,"{'BikeParking': 'False', 'Caters': 'False', 'O...","Coffee & Tea, Food","{'Monday': '5:30-19:0', 'Tuesday': '5:30-19:0'..."
1543,lo2VbA7Lwzu3i23pQpmzlw,Starbucks,"1125 S Black Horse Pike, Space 575",Gloucester Township,NJ,08012,39.78403,-75.05207,3.0,15,1,"{'BikeParking': 'True', 'RestaurantsPriceRange...","Food, Coffee & Tea","{'Monday': '0:0-0:0', 'Tuesday': '10:30-20:0',..."


In [9]:
del businessDuppedDf

Verificamos la existencia de nulos de businessDf.

In [10]:
utils.dataType(businessDf)

,name,data_type,not_null_%,null_%,not_null,null
0,business_id,[<class 'str'>],100.0000,0.0000,150346,0
1,name,[<class 'str'>],100.0000,0.0000,150346,0
2,address,[<class 'str'>],100.0000,0.0000,150346,0
3,city,[<class 'str'>],100.0000,0.0000,150346,0
4,state,"[<class 'float'>, <class 'str'>]",99.9980,0.0020,150343,3
5,postal_code,[<class 'str'>],100.0000,0.0000,150346,0
6,latitude,[<class 'float'>],100.0000,0.0000,150346,0
7,longitude,[<class 'float'>],100.0000,0.0000,150346,0
8,stars,[<class 'float'>],100.0000,0.0000,150346,0
9,review_count,[<class 'int'>],100.0000,0.0000,150346,0


Eliminamos columnas que no se utilizarán en el proyecto.

In [11]:
businessDf = businessDf.drop(columns=['attributes', 'attributes', 'hours','address'])

Filtramos las categorías para quedarnos únicamente con los datos relacionados a entidades bancarias.

In [12]:
# Separamos las categorías y les asignamos una fila a cada una
categoryDf = businessDf[['business_id', 'categories']].assign(categories = businessDf['categories'].str.split(', ')).explode('categories')
# Filtramos las categorías por 'Banks & Credit Unions'
categoryDf = categoryDf[categoryDf['categories'] == 'Banks & Credit Unions']
# Filtramos el dataframe con base a los bancos.
businessDf = businessDf[businessDf['business_id'].isin(categoryDf['business_id'].unique())].reset_index(drop=True)

Verificamos que no existan duplicados.

In [13]:
businessDf[businessDf.duplicated('business_id') == True].shape

(0, 11)

Tenemos la información de 749 comercios bancarios.

In [14]:
businessDf.shape

(749, 11)

Corregimos los estados de los bancos.

In [21]:
businessDf['state'] = businessDf['postal_code'].apply(utils.stateFromZip)
businessDf = businessDf[businessDf['state'] != 'Estado no encontrado']

Verificamos que la información se encuentre completa y guardamos en parquet

In [23]:
utils.dataType(businessDf)

,name,data_type,not_null_%,null_%,not_null,null
0,business_id,[<class 'str'>],100.0,0.0,738,0
1,name,[<class 'str'>],100.0,0.0,738,0
2,city,[<class 'str'>],100.0,0.0,738,0
3,state,[<class 'str'>],100.0,0.0,738,0
4,postal_code,[<class 'str'>],100.0,0.0,738,0
5,latitude,[<class 'float'>],100.0,0.0,738,0
6,longitude,[<class 'float'>],100.0,0.0,738,0
7,stars,[<class 'float'>],100.0,0.0,738,0
8,review_count,[<class 'int'>],100.0,0.0,738,0
9,is_open,[<class 'int'>],100.0,0.0,738,0


In [24]:
table = pa.Table.from_pandas(businessDf)
pq.write_table(table, '../data/Yelp/business.snappy.parquet', compression='SNAPPY')

In [25]:
businessDf.to_csv('../data/Cloud_Upload/Yelp/business/business.csv',encoding = 'utf-8', index = False)

In [15]:
# businessDf.to_parquet('../data/Yelp/business.parquet', index = False)

In [16]:
del businessDf

<h2>Checkin

Cargamos el archivo checkin.json

In [17]:
checkinDf = utils.loadFile('../data/Yelp\checkin.json')

In [18]:
checkinDf.head()

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."


Revisamos la existencia de datos nulos.

In [19]:
utils.dataType(checkinDf)

,name,data_type,not_null_%,null_%,not_null,null
0,business_id,[<class 'str'>],100.0,0.0,131930,0
1,date,[<class 'str'>],100.0,0.0,131930,0


Verificamos la existencia de duplicados.

In [20]:
checkinDf[checkinDf.duplicated('business_id') == True].shape

(0, 2)

Filtramos los valores de 'business_id' quedándonos con los datos relacionados a entidades bancarias.

In [21]:
checkinDf = checkinDf[checkinDf['business_id'].isin(categoryDf['business_id'].unique())].reset_index(drop=True)
checkinDf.shape

(700, 2)

Separamos la columna 'dates' creando una fila por cada check in.

In [22]:
checkinDf = checkinDf.assign(date = checkinDf['date'].str.split(', ')).explode('date').reset_index(drop=True)

Transformamos la columna 'date' a datetime y creamos dos columnas, una para la fecha y otra para la hora.

In [23]:
checkinDf['date'] = checkinDf['date'].apply(pd.to_datetime, format='%Y-%m-%d %H:%M:%S', errors='coerce')
checkinDf['hour'] = checkinDf['date'].dt.hour
checkinDf['time'] = checkinDf['date'].dt.time
checkinDf['date'] = checkinDf['date'].dt.date


Realizamos una última revisión y guardamos en parquet.

In [24]:
utils.dataType(checkinDf)

,name,data_type,not_null_%,null_%,not_null,null
0,business_id,[<class 'str'>],100.0,0.0,25736,0
1,date,[<class 'datetime.date'>],100.0,0.0,25736,0
2,hour,[<class 'int'>],100.0,0.0,25736,0
3,time,[<class 'datetime.time'>],100.0,0.0,25736,0


In [25]:
table = pa.Table.from_pandas(checkinDf)
pq.write_table(table, '../data/Yelp/checkin.snappy.parquet', compression='SNAPPY')

In [26]:
checkinDf.to_csv('../data/Cloud_Upload/Yelp/checkin/checkin.csv', index = False)

In [27]:
# checkinDf.to_parquet('../data/Yelp/checkin.parquet', index = False)

Borramos el dataFrame para ahorrar espacio en disco.

In [28]:
del checkinDf

<h2>Review

In [29]:
reviewDf = utils.loadFile('../data/Yelp/review.json')

In [30]:
reviewDf.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [31]:
reviewDf = reviewDf[reviewDf['business_id'].isin(categoryDf['business_id'].unique())].reset_index(drop=True)

In [32]:
userIdDf = pd.DataFrame({'user_id': reviewDf['user_id'].unique()})

In [33]:
reviewDf['date'] = reviewDf['date'].apply(pd.to_datetime, format='%Y-%m-%d %H:%M:%S', errors='coerce')
reviewDf['hour'] = reviewDf['date'].dt.hour
reviewDf['time'] = reviewDf['date'].dt.time
reviewDf['date'] = reviewDf['date'].dt.date

In [34]:
utils.dataType(reviewDf)

,name,data_type,not_null_%,null_%,not_null,null
0,review_id,[<class 'str'>],100.0,0.0,7924,0
1,user_id,[<class 'str'>],100.0,0.0,7924,0
2,business_id,[<class 'str'>],100.0,0.0,7924,0
3,stars,[<class 'int'>],100.0,0.0,7924,0
4,useful,[<class 'int'>],100.0,0.0,7924,0
5,funny,[<class 'int'>],100.0,0.0,7924,0
6,cool,[<class 'int'>],100.0,0.0,7924,0
7,text,[<class 'str'>],100.0,0.0,7924,0
8,date,[<class 'datetime.date'>],100.0,0.0,7924,0
9,hour,[<class 'int'>],100.0,0.0,7924,0


In [35]:
reviewDf.duplicated().any()

False

In [36]:
reviewDf.duplicated('review_id').any()

False

In [37]:
table = pa.Table.from_pandas(reviewDf)
pq.write_table(table, '../data/Yelp/review.snappy.parquet', compression='SNAPPY')

In [38]:
reviewDf.to_csv('../data/Cloud_Upload/Yelp/review/review.csv',encoding='utf-8', index = False)

In [39]:
# reviewDf.to_parquet('../data/Yelp/review.parquet', index = False)

In [40]:
del reviewDf

<h2>Tip

In [41]:
tipDf = utils.loadFile('../data/Yelp/tip.json')

In [42]:
tipDf.head()

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0


In [43]:
tipDf = tipDf[tipDf['business_id'].isin(categoryDf['business_id'].unique())].reset_index(drop=True)

In [44]:
tipDf['date'] = tipDf['date'].apply(pd.to_datetime, format='%Y-%m-%d %H:%M:%S', errors='coerce')
tipDf['hour'] = tipDf['date'].dt.hour
tipDf['time'] = tipDf['date'].dt.time
tipDf['date'] = tipDf['date'].dt.date

In [45]:
utils.dataType(tipDf)

,name,data_type,not_null_%,null_%,not_null,null
0,user_id,[<class 'str'>],100.0,0.0,997,0
1,business_id,[<class 'str'>],100.0,0.0,997,0
2,text,[<class 'str'>],100.0,0.0,997,0
3,date,[<class 'datetime.date'>],100.0,0.0,997,0
4,compliment_count,[<class 'int'>],100.0,0.0,997,0
5,hour,[<class 'int'>],100.0,0.0,997,0
6,time,[<class 'datetime.time'>],100.0,0.0,997,0


In [46]:
tipDf.duplicated().any()

False

In [47]:
table = pa.Table.from_pandas(tipDf)
pq.write_table(table, '../data/Yelp/tip.snappy.parquet', compression='SNAPPY')

In [48]:
tipDf.to_csv('../data/Cloud_Upload/Yelp/tip/tip.csv',encoding='utf-8', index = False)

In [49]:
# tipDf.to_parquet('../data/Yelp/tip.parquet', index = False)

In [50]:
del tipDf

<h2>User

In [51]:
userDf = utils.loadFile('../data/Yelp/user.parquet')

In [52]:
userDf.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0


In [53]:
userDf = userDf[userDf['user_id'].isin(userIdDf['user_id'].unique())].reset_index(drop=True)

In [54]:
userDf = userDf.drop_duplicates()

In [55]:
userDf.duplicated('user_id').any()

False

In [56]:
userDf = userDf.drop(columns=['elite'])

In [57]:
userDf['yelping_since'] = userDf['yelping_since'].apply(pd.to_datetime, format='%Y-%m-%d %H:%M:%S', errors='coerce')
userDf['yelping_since'] = userDf['yelping_since'].dt.date

In [58]:
utils.dataType(userDf)

,name,data_type,not_null_%,null_%,not_null,null
0,user_id,[<class 'str'>],100.0,0.0,6910,0
1,name,[<class 'str'>],100.0,0.0,6910,0
2,review_count,[<class 'int'>],100.0,0.0,6910,0
3,yelping_since,[<class 'datetime.date'>],100.0,0.0,6910,0
4,useful,[<class 'int'>],100.0,0.0,6910,0
5,funny,[<class 'int'>],100.0,0.0,6910,0
6,cool,[<class 'int'>],100.0,0.0,6910,0
7,friends,[<class 'str'>],100.0,0.0,6910,0
8,fans,[<class 'int'>],100.0,0.0,6910,0
9,average_stars,[<class 'float'>],100.0,0.0,6910,0


In [59]:
table = pa.Table.from_pandas(userDf)
pq.write_table(table, '../data/Yelp/user.snappy.parquet', compression='SNAPPY')

In [60]:
userDf.to_csv('../data/Cloud_Upload/Yelp/user/user.csv',encoding='utf-8', index = False)

In [61]:
del userDf